In [1]:
import pandas as pd
import csv
import collections
import re
import pickle
import numpy as np
#import nltk
import edlib

In [2]:
fileNeop = "../data/RTRT_NEOPLASI_corr.csv"
fileIsto = "../data/ISTOLOGIE_corr.csv" 
fileIcdo = "icdo-3-dict-en.p"

fileSaveDistances = "editDistances.npz"

In [3]:
encoding = 'iso-8859-1'

In [4]:
icdo = pickle.load(open(fileIcdo, 'rb'))

# Load neop

In [5]:
dfNeop = pd.read_csv(fileNeop, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False,
                     usecols = ['id_neopl', 'sede_icdo3',
                                'morfologia_icdo3', 'inserimento_data_neoplasi'],
                     dtype={'id_neopl':pd.np.float64, 'sede_icdo3':pd.np.str, 
                            'morfologia_icdo3':pd.np.str, 'inserimento_data_neoplasi':pd.np.str},
                    #parse_dates=['inserimento_data_neoplasi'], 
                    #date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
                    )

In [6]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [7]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [8]:
dfNeop = dfNeop.dropna('index')

In [9]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [10]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [11]:
dfNeop['inserimento_data_neoplasi'] =  pd.to_datetime(dfNeop['inserimento_data_neoplasi'], format='%Y-%m-%d %H:%M:%S')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [13]:
dfNeop.dtypes

id_neopl                            float64
inserimento_data_neoplasi    datetime64[ns]
sede_icdo3                           object
morfologia_icdo3                     object
dtype: object

In [14]:
dfNeop = dfNeop.sort_values(by=['inserimento_data_neoplasi'])

In [15]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
107276,108110.0,1990-04-20 00:00:00,C443,80003
78576,79172.0,1990-04-20 00:00:00,C341,80003
78595,79191.0,1990-04-24 00:00:00,C329,80703
78602,79198.0,1990-04-24 00:00:00,C509,85203
78601,79197.0,1990-04-24 00:00:00,C710,94003
...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001
295873,346866.0,2014-03-10 12:26:26,C169,81403


remove invalid sites

In [16]:
dfNeop['correctSede'] = dfNeop.sede_icdo3.map(lambda s: len(s)==4 and s[0] == "C" and int(s[1:3]) in icdo.keys())

In [17]:
dfNeop = dfNeop[dfNeop.correctSede]

In [18]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede
107276,108110.0,1990-04-20 00:00:00,C443,80003,True
78576,79172.0,1990-04-20 00:00:00,C341,80003,True
78595,79191.0,1990-04-24 00:00:00,C329,80703,True
78602,79198.0,1990-04-24 00:00:00,C509,85203,True
78601,79197.0,1990-04-24 00:00:00,C710,94003,True
...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True
309849,346856.0,2014-02-14 11:53:40,C424,97651,True
309850,346857.0,2014-02-14 12:00:33,C619,80001,True
295873,346866.0,2014-03-10 12:26:26,C169,81403,True


In [19]:
dfNeop['y'] = dfNeop.sede_icdo3.map(lambda s: int(s[1:3]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
dfNeop['yM'] = dfNeop.morfologia_icdo3.map(lambda m: int(m[:4]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y,yM
107276,108110.0,1990-04-20 00:00:00,C443,80003,True,44,8000
78576,79172.0,1990-04-20 00:00:00,C341,80003,True,34,8000
78595,79191.0,1990-04-24 00:00:00,C329,80703,True,32,8070
78602,79198.0,1990-04-24 00:00:00,C509,85203,True,50,8520
78601,79197.0,1990-04-24 00:00:00,C710,94003,True,71,9400
...,...,...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True,50,8500
309849,346856.0,2014-02-14 11:53:40,C424,97651,True,42,9765
309850,346857.0,2014-02-14 12:00:33,C619,80001,True,61,8000
295873,346866.0,2014-03-10 12:26:26,C169,81403,True,16,8140


# Load isto

In [22]:
dfIsto = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [23]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [24]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfIsto['notizie'] = dfIsto.notizie.map(nullToEmpty)
dfIsto['diagnosi'] = dfIsto.diagnosi.map(nullToEmpty)
dfIsto['macroscopia'] = dfIsto.macroscopia.map(nullToEmpty)

In [25]:
dfIsto = dfIsto.dropna('index')

In [26]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril..."
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.
...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...


In [27]:
dfIsto['text'] = dfIsto[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
dfIsto['text'] = dfIsto['text'].astype('unicode')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
dfIsto['text'] = dfIsto['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...
...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...


In [31]:
dfIsto['correctText'] = dfIsto.text.map(lambda t: len(t) > 0)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
dfIsto = dfIsto[dfIsto.correctText]

In [33]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [34]:
dfIsto = dfIsto.drop_duplicates(['text'])

In [35]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [36]:
dfIsto = dfIsto.drop_duplicates(['id_neopl'], keep=False)

In [37]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
684,685.0,341032.0,,CARCINOMA BASOCELLULARE; MARGINI DI SEZIONE LI...,"1-2) Losanga di cute di cm 2.5, lesione irrego...",CARCINOMA BASOCELLULARE ; MARGINI DI SEZIONE L...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


# Merge

In [38]:
dfMerge = pd.merge(dfIsto, dfNeop, on='id_neopl', validate="one_to_one")

In [39]:
dfMerge

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y,yM
0,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True,2008-03-06 14:14:39,C449,80903,True,44,8090
1,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True,2012-11-14 10:01:56,C449,80903,True,44,8090
2,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True,2007-02-09 10:07:13,C779,96513,True,77,9651
3,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True,2012-10-10 10:19:28,C209,81401,True,20,8140
4,685.0,341032.0,,CARCINOMA BASOCELLULARE; MARGINI DI SEZIONE LI...,"1-2) Losanga di cute di cm 2.5, lesione irrego...",CARCINOMA BASOCELLULARE ; MARGINI DI SEZIONE L...,True,2012-10-10 14:21:28,C449,80903,True,44,8090
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85170,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True,2013-10-18 16:23:09,C443,80903,True,44,8090
85171,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True,2012-09-10 16:00:08,C186,80003,True,18,8000
85172,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True,2012-10-15 11:56:30,C329,80703,True,32,8070
85173,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True,2012-11-12 10:35:31,C447,87432,True,44,8743


# Final

In [40]:
dfFinal = dfMerge[['inserimento_data_neoplasi','text', 'y', 'yM']]

In [41]:
dfFinal = dfFinal.rename(columns={"inserimento_data_neoplasi": "data"})

In [42]:
dfFinal = dfFinal.sort_values(by=['data'])

In [43]:
dfFinal

,data,text,y,yM
61615,1992-04-13 00:00:00,METASTASI EPATICHE DA SOSPETTA NEOPLASIA COLON...,16,8140
70999,1992-04-15 00:00:00,PSA : 1160 ; AREA IPOECOGENA LOBO SINISTRO ; I...,75,8140
46748,1996-03-13 00:00:00,CARCINOMA DUTTALE INFILTRANTE ( NOS ) ( G2 ) D...,50,8501
17030,1997-03-04 00:00:00,LESIONE PIGMENTATA DISCROMICA E PALPABILE OD :...,44,8743
67580,1997-09-15 00:00:00,AREE DI ADENOCARCINOMA PROSTATICO A MEDIO GRAD...,23,8560
...,...,...,...,...
84732,2014-02-13 08:51:32,CARCINOMA DUTTALE INFILTRANTE ( G2 ) DELLA MAM...,50,8500
47068,2014-02-13 11:20:50,"CARCINOMA DUTTALE IN SITU NAS , PRESENZA DI CA...",50,8500
78074,2014-02-14 12:00:33,1 - AGOBIOPSIA PROSTATICA DELLA LUNGHEZZA COMP...,61,8000
1000,2014-03-10 12:26:26,CA . GASTRICO . ADENOCARCINOMA GASTRICO MODERA...,16,8140


# Calculate distance

In [44]:
corpus = {
        'text':dfFinal['text'].values,
        'sede1':dfFinal['y'].values,
        'morfo1':dfFinal['yM'].values,
    }

In [45]:
dist = np.zeros((len(corpus['text']), len(corpus['text'])), dtype=np.uint16)
numDocs = len(corpus['text'])
numDocs = 100
for i in range(numDocs):
    for j in range(i+1, numDocs):
        if j % 1000 == 0:
            print("{}/{} - {}/{}          ".format(i, numDocs, j, numDocs), end='\r', flush=True)
        #dist[i,j] = dist[j,i] = nltk.edit_distance(corpus['text'][i], corpus['text'][j])
        dist[i,j] = dist[j,i] = edlib.align(corpus['text'][i], corpus['text'][j])['editDistance']
print("{}/{} - {}/{}        ".format(i, numDocs, j, numDocs), flush=True)

99/100 - 99/100        


In [46]:
dist

array([[  0, 593, 229, ...,   0,   0,   0],
       [593,   0, 524, ...,   0,   0,   0],
       [229, 524,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint16)

In [47]:
corpus['distance'] = dist

# write

In [53]:
np.savez_compressed(fileSaveDistances, text=corpus['text'], sede1=corpus['sede1'], morfo1=corpus['morfo1'], distance=corpus['distance'])